In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version

spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,693 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,837 k

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)


In [12]:
# 2. Create a temporary view of the DataFrame.
home_data=spark.read.csv(
    SparkFiles.get("home_sales_revised.csv"),
    header=True, inferSchema=True, quote="\"", escape="\"")
home_data.show()
home_data.createOrReplaceTempView('home_sales')

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [14]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) as YEAR, ROUND(AVG(price)) as AVG_PRICE
FROM  home_sales
WHERE bedrooms=4
GROUP BY YEAR
ORDER BY YEAR DESC
""").show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2022| 296364.0|
|2021| 301819.0|
|2020| 298354.0|
|2019| 300264.0|
+----+---------+



In [16]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price)) as AVG_PRICE
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3
GROUP BY date_built
ORDER BY date_built DESC
""").show()

+----------+---------+
|date_built|AVG_PRICE|
+----------+---------+
|      2017| 292677.0|
|      2016| 290555.0|
|      2015| 288770.0|
|      2014| 290852.0|
|      2013| 295962.0|
|      2012| 293683.0|
|      2011| 291117.0|
|      2010| 292860.0|
+----------+---------+



In [17]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price)) as AVG_PRICE
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
GROUP BY date_built
ORDER BY date_built DESC
""").show()


+----------+---------+
|date_built|AVG_PRICE|
+----------+---------+
|      2017| 280318.0|
|      2016| 293965.0|
|      2015| 297610.0|
|      2014| 298265.0|
|      2013| 303677.0|
|      2012| 307540.0|
|      2011| 276554.0|
|      2010| 285010.0|
+----------+---------+



In [27]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price),2) as AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price)>=350000
ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| AVG_PRICE|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6591765880584717 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [35]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price),2) as AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price)>=350000
ORDER BY view DESC
""")

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.045246124267578125 seconds ---


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_data.write.partitionBy("date_built").parquet('parquet_delay_basic', mode='overwrite')

In [31]:
# 11. Read the parquet formatted data.
p_home_sales=spark.read.parquet('parquet_delay_basic')

In [32]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView('p_home_sales')

In [34]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price),2) as AVG_PRICE
FROM p_home_sales
GROUP BY view
HAVING AVG(price)>=350000
ORDER BY view DESC
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.030943632125854492 seconds ---


In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False